<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_genai_hackathon.gif" width="700px">

<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_titulo.png" width="900px">

## Título: Gerador de Artigos

| ITEM | DESCRIÇÃO |
| -- | -- |
| Indústria: | Digital |
| Departamento: | Marketing |
| Tipo de Solução: | Gerador de Conteúdo |

%md
<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_grupo.png" width="900px">


### GRUPO 09 - Mecânicos de Busca

| # | Nome do Integrante | Empresa | e-mail |
| -- | -- | -- | -- |
| 1 | Leonardo Machado | Consumidor Positivo | leonardo.machado@consumidorpositivo.com.br |
| 2 | Francisco Cidade | Consumidor Positivo | francisco.cidade@consumidorpositivo.com.br |
| 3 | William Nicolau | Consumidor Positivo | william.nicolau@consumidorpositivo.com.br |
| 4 | Vinicio Braga | Consumidor Positivo | vinicio.braga@consumidorpositivo.com.br |
| 5 | Virginia Costa | Volkswagen | virginia.costa@volkswagen.com.br | 
| 6 | Caroline Melo | Volkswagen | caroline.melo@volkswagen.com.br | 

<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_contexto.png" width="900px">

#### Cenário Atual

Empresas de todos os tamanhos, incluindo negócios individuais, gastam uma grande quantidade de recursos com a estratégia de SEO - Search Engine Optimization.

O objetivo do SEO é gerar artigos de blogs que "rankeiam" no Google - assim, quando pessoas buscam um termo do tipo "como fazer um bolo de chocolate", o artigo que aparece em primeiro lugar recebe um grande número de acessos "orgânicos".

#### Dores / Necessidades do Negócio

Para gerar esses artigos, geralmente é preciso contratar redatores, editores e revisores, que são responsáveis por pesquisar um tema e produzir conteúdo sobre o tema - o processo de pesquisa, escrita e revisão pode ser demorado e caro.

Diminuir a quantidade de recursos necessários para gerar conteúdo novo para um site e acelerar o processo pode ser uma grande alavanca para "criar" volume de acessos em um site.


<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_proposito.png" width="900px">

#### Objetivo da Solução Proposta

- Utilizar uma combinação de tecnologias (Google Search Console, modelos de linguagem e prompt engineering) para descobrir os "melhores" textos sobre um certo assunto (os 10 primeiros resultados do Google, por exemplo);
- Criar uma "base de conhecimento" utilizando esses textos para evitar que o modelo de linguagem produza conteúdo falso;
- Utilizar o modelo de linguagem para produzir um novo texto, utilizando o conteúdo obtido, com o objetivo de rankear na primeira página do Google.

#### Benefícios

- Redução de custos para geração de conteúdo
- Melhoria da eficiência de processo de geração de conteúdo
- Potencial de aumento de usuários vindos de canal "orgânico" no site
- Desvio do foco de geração de conteúdo para "curadoria" do conteúdo - processo centrado na revisão

<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_arquitetura.png" width="900px">


### Arquitetura

* Provedor Cloud: AWS
* Cluster / DB Runtime: 14.3.x LTS - r5d.large
* Bibliotecas utilizadas: `openai` `google-api-python-client` `html2text` `bs5` `requests`

### Técnicas Utilizadas

* Web Scrapping
* Prompt Engineering (external-model: openai - `gpt-3.5-turbo`)


<img src="https://raw.githubusercontent.com/Databricks-BR/genai_hackathon/main/images/head_referencias.png" width="900px">


#### Referências:
* [Introdução ao DBRX LLM foundation model Databricks](https://www.databricks.com/blog/introducing-dbrx-new-state-art-open-llm)
* [Documentação do Gen AI Databricks](https://docs.databricks.com/en/generative-ai/generative-ai.html)
* [Foundation LLM Models APIs](https://docs.databricks.com/en/machine-learning/foundation-models/index.html#pay-per-token-foundation-model-apis)
* [OpenAI API Reference](https://platform.openai.com/docs/api-reference/introduction)
* [Google Search Console API Reference](https://developers.google.com/webmaster-tools/v1/api_reference_index)


# Execução do Notebook

#### Instalando dependências

In [0]:
%pip install openai google-api-python-client html2text

In [0]:
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
from openai import OpenAI

import html2text
import requests

#### Definindo funções auxiliares

In [0]:
def extract_website(url):
  response = requests.get(url)
  html_content = response.text

  soup = BeautifulSoup(html_content, "html.parser")

  return (html2text.html2text(soup.body.prettify()))

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

#### Pré-requisitos

Para utilizar esse código, você vai precisar:
- De uma chave de API de um projeto Google
- De um "custom search ID" do Google Search Console
- De uma chave de API da OpenAI

In [0]:
api_key = dbutils.secrets.get("google_search", "api_key")
cse_id = dbutils.secrets.get("google_search", "cse_id")
open_ai_key = dbutils.secrets.get("openai", "api_key")

In [0]:
dbutils.widgets.text("search_text", defaultValue = "como limpar meu nome", label = "Search Text")

search_text = dbutils.widgets.get("search_text")

print(f"Texto utilizado para busca: `{search_text}`")

In [0]:
results = google_search(search_text, api_key, cse_id)

total_links = []

for item in results['items']:
    total_links.append(item["link"])

display(total_links)

# total_links = [
#   "https://www.serasa.com.br/limpa-nome-online/blog/como-limpar-o-nome-na-serasa/",
#   "https://www.acordocerto.com.br/blog/como-limpar-seu-nome/",
#   "https://www.serasa.com.br/limpa-nome-online/",
#   "https://meubolsoemdia.com.br/Materias/nome-sujo-como-limpar-nome",
#   "https://quiteja.com.br/blog/como-limpar-nome/",
#   "https://www.spcbrasil.org.br/consumidor/negociar",
#   "https://ajuda.serasa.com.br/hc/pt-br/articles/360004225412-Quero-limpar-meu-nome-Por-onde-come%C3%A7o",
#   "https://economia.uol.com.br/noticias/redacao/2023/01/12/como-limpar-nome-sujo.htm",
#   "https://www.jusbrasil.com.br/artigos/exclusao-de-registros-de-antecedentes-criminais-como-limpar-meu-nome/1444443728",
#   "https://www.creditas.com/exponencial/como-limpar-nome-sujo/",
# ]

#### Buscando o conteúdo dos sites

In [0]:
contents = []
errs = []
for link in total_links:
    try:
        contents.append(extract_website(link))
    except Exception as e:
        errs.append(e)

print('err count:', len(errs))

In [0]:
print(contents[3][0:1000])

In [0]:
client = OpenAI(api_key=open_ai_key)

In [0]:
my_assistant = client.beta.assistants.create(
    instructions=f"""
    Eu vou te enviar uma lista de artigos extraídos da internet, que respondem à seguinte pergunta: \"{search_text}\"
    Preciso que você faça o seguinte:
    1. Produza um novo artigo, de tamanho semelhante aos artigos enviados.
    2. Utilize apenas o conhecimento extraído dos artigos para gerar o novo texto - não produza informação nova não-verificada.
    3. Consolide os principais pontos em comum de todos os artigos.
    4. Escreva o artigo em uma linguagem acessível.
    5. Otimize o artigo escrito para rankear em SEO.
    6. Não cometa plágio - não copie diretamente conteúdo ou blocos de conteúdo da base de conhecimento. Seja transformativo na escrita.
    Gere o conteúdo APENAS depois de receber a mensagem "Comece a gerar o novo artigo."
    """,
    name="Article Writer",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo",
)

In [0]:
msg_thread = client.beta.threads.create()
i = 0
for content in contents:
    i = i + 1
    msg = f" Texto {i}: `{content}`"
    thread_message = client.beta.threads.messages.create(
        msg_thread.id,
        role="user",
        content=msg,
    )

final_msg = client.beta.threads.messages.create(
  msg_thread.id,
  role="user",
  content="Comece a gerar o novo artigo."
)

In [0]:
thread_run = client.beta.threads.runs.create(
    thread_id=msg_thread.id,
    assistant_id=my_assistant.id,
    stream=True
)
texto_final = ""
for chunk in thread_run:
  try:
    if chunk.data.delta.content is not None:
      texto_final+= chunk.data.delta.content[0].text.value
      print(texto_final, end="")
  except:
    ...

In [0]:
from IPython.display import display, Markdown
display(Markdown(texto_final))

#### Evoluções
Para tornar o produto ainda melhor, enxergamos os seguintes caminhos:
- Utilização de algum modelo de linguagem ou ferramenta externa de detecção de plágio
  - Alguns *approaches* diferentes funcionariam aqui, como por exemplo passar o texto em algumas camadas de prompts para garantir uma maior randomização do output
- Utilizar um modelo `fine-tuning` para treinar o modelo em um estilo de escrita
  - Uma empresa grande que possui uma grande base de conteúdo escrito por redatores, pode acabar adotando um padrão de estilo de escrita (linguagem mais formal ou informal, mais acessível, etc.) - copiar esse estilo pode tornar a experiência de artigos de um site mais coesa e unificada
- Bater o modelo contra um "verificador de geração de AI"
  - É importante utilizar alguma estratégia para minimizar a "chance de detecção" do texto ser escrito por uma IA, para maximizar as chances de rankear no Google